<a href="https://colab.research.google.com/github/amithpaul/LSTMPdM/blob/main/LSTMtrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: import numpy as np
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dense, Input
# from tensorflow.keras.callbacks import EarlyStopping
# install libraries for these





In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib as plt
tf.config.set_visible_devices([], 'GPU')

# Clear the Keras session to reset the model's state
K.clear_session()

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Load preprocessed dataset
data = np.load("processed_datasets/scaled_minmax.npy").astype(np.float32)  # Ensure float32
print("Shape of loaded data:", data.shape)
input_dim = data.shape[1]  # Number of features

Shape of loaded data: (1284398, 38)


In [8]:
# Create sequences
seq_length = 180  # Adjust based on your dataset
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i : i + seq_length])
    return np.array(sequences)

X_train = create_sequences(data, seq_length)

In [10]:
# LSTM Autoencoder
timesteps, features = X_train.shape[1], X_train.shape[2]


input_layer = Input(shape=(timesteps, features))
masked_input = Masking(mask_value=-1)(input_layer)
encoded = LSTM(64, activation="relu", return_sequences=True)(masked_input)
encoded = Dropout(0.2)(encoded)  # NEW DROPOUT
encoded = LSTM(32, activation="relu", return_sequences=True)(encoded)

decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(features, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse",jit_compile=True)

/Users/amith_abhay/Documents/PdM/.conda/lib/python3.9/site-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(


In [13]:
class PlotLossCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        plt.ion()  # Turn on interactive mode

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

        # Clear previous plot
        plt.clf()
        plt.plot(self.losses, label='Training Loss')
        plt.plot(self.val_losses, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.pause(0.1)  # Pause to update the figure

        print(f"Epoch {epoch+1}: loss = {logs.get('loss'):.5f}, val_loss = {logs.get('val_loss'):.5f}")


In [ ]:
# Train the model
autoencoder.fit(
    X_train, X_train,
    epochs=200, batch_size=256,
    validation_split=0.1,
    callbacks=[early_stop, PlotLossCallback]
)

In [ ]:
# Save the model
autoencoder.save("lstm_autoencoderV2.keras")